In this notebook, we compute the AR(1) innovations on both indexes of the aricle. We compare them to the already computed innovations to check if the results are the same. This will be useful as we will have to compute the AR(1)  innovations of an index that is yet to be built.

We deal with data using pandas, and plotly for visualization purposes.

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objects as go

In [9]:
data = pd.read_excel('EGLKS_data.xlsx')
data['Date'] = data['Date'].apply(lambda x : pd.Timestamp(str(x) + '01'))

In [182]:
wsj = data[['Date','wsj','wsj_AR1_Innovation']].dropna(subset = ['wsj'])
chneg = data[['Date','chneg','chneg_AR1_innovation']].dropna(subset = ['chneg'])

In [198]:
chneg

,Date,chneg,chneg_AR1_innovation
293,2008-06-01,0.006646,NaN
294,2008-07-01,0.007067,0.001682
295,2008-08-01,0.003981,-0.001713
296,2008-09-01,0.005558,0.002119
297,2008-10-01,0.004507,-0.000085
...,...,...,...
408,2018-01-01,0.001457,-0.000041
409,2018-02-01,0.000954,-0.000643
410,2018-03-01,0.000807,-0.000422
411,2018-04-01,0.001055,-0.000067


# Plotting

Let's first plot the data we have. We have the evolution of the WSJ index since 1984 and the one of the Climate Change News since 2008.

In [91]:
labels={"chneg": "Climate Change News"}
fig = go.Figure()
fig.add_trace(go.Scatter(mode = 'lines', x = data['Date'], y = data['wsj'], name = 'wsj'))
fig.add_trace(go.Scatter(mode = 'lines', x = data['Date'], y = data['chneg'], name = 'Climate Change News', marker=dict(color='green')))
fig.update_yaxes(title = 'Climate Change Index',
    tickprefix="$", showgrid=True)
fig.update_layout( title = 'Climate Change Index evolution' ,
    font_family="Rockwell")

# Creating and fitting an AR(1) model

In [13]:
model = ARIMA(wsj['wsj'], order = (1,0,0))
results = model.fit()

In [14]:
print(results.summary())

                               SARIMAX Results                                
Dep. Variable:                    wsj   No. Observations:                  402
Model:                 ARIMA(1, 0, 0)   Log Likelihood                2025.873
Date:                Sat, 22 Oct 2022   AIC                          -4045.747
Time:                        16:06:40   BIC                          -4033.758
Sample:                             0   HQIC                         -4041.000
                                - 402                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0054      0.000     20.788      0.000       0.005       0.006
ar.L1          0.5864      0.024     24.763      0.000       0.540       0.633
sigma2      2.454e-06   8.35e-08     29.392      0.0

# Forceasting

We make one step ahead predictions, beggining from the first value of the series

In [187]:
forecast = results.get_prediction(start=-401)

In [188]:
print(forecast.predicted_mean)

1      0.004349
2      0.004640
3      0.004503
4      0.004230
5      0.004139
         ...   
397    0.005335
398    0.005006
399    0.005351
400    0.005157
401    0.005471
Name: predicted_mean, Length: 401, dtype: float64


In [201]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= wsj['Date'], y = wsj['wsj'], mode="lines", name = 'Index', opacity = 0.6))
fig.add_trace(go.Scatter(x = wsj['Date'].iloc[1:], y = forecast.predicted_mean, name = 'Forecast'))
fig.update_layout(title = 'Forecasted WSJ index with an AR(1) model')
fig.update_yaxes(title = 'WSJ Index')
fig.update_xaxes(title = 'Date')

# Comparing the results of AR(1) models : innovations

In [196]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= wsj['Date'], y = wsj['wsj_AR1_Innovation'].dropna(), mode="lines", name = 'Pre computed'))
fig.add_trace(go.Scatter(x= wsj['Date'], y = wsj['wsj'].iloc[1:] - forecast.predicted_mean, mode="lines", name = 'computed', opacity = 0.5))


In [197]:
px.line(x = wsj['Date'] ,y = -wsj['wsj_AR1_Innovation']- (forecast.predicted_mean - wsj['wsj'].iloc[1:]), title = 'Difference between results : only computation errors')


# CHNEG

Let's do the same for the other index 

In [199]:
model2 = ARIMA(chneg['chneg'], order = (1,0,0))
results2 = model2.fit()
print(results2.summary())

                               SARIMAX Results                                
Dep. Variable:                  chneg   No. Observations:                  120
Model:                 ARIMA(1, 0, 0)   Log Likelihood                 687.587
Date:                Sun, 23 Oct 2022   AIC                          -1369.173
Time:                        21:02:17   BIC                          -1360.811
Sample:                             0   HQIC                         -1365.777
                                - 120                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0022      0.001      4.216      0.000       0.001       0.003
ar.L1          0.8140      0.043     19.014      0.000       0.730       0.898
sigma2      6.042e-07   5.09e-08     11.875      0.0

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



In [200]:
forecast2 = results2.get_prediction(start=-119)

In [203]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= chneg['Date'], y = chneg['chneg'].dropna(), mode="lines", name = 'Index', opacity = 0.6))
fig.add_trace(go.Scatter(x = chneg['Date'].iloc[1:], y = forecast2.predicted_mean, name = 'Forecast'))
fig.update_layout(title = 'Forecasted CHNEG index with an AR(1) model')
fig.update_yaxes(title = 'CHNEG Index')

fig.update_xaxes(title = 'Date')

In [212]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= chneg['Date'], y = chneg['chneg_AR1_innovation'], mode="lines", name = 'Pre computed'))
fig.add_trace(go.Scatter(x= chneg['Date'], y = chneg['chneg'] - forecast2.predicted_mean, mode="lines", name = 'Computed'))


In [211]:
chneg['chneg'] - forecast2.predicted_mean

293         NaN
294    0.001239
295   -0.002190
296    0.001900
297   -0.000436
         ...   
408   -0.000037
409   -0.000650
410   -0.000388
411   -0.000020
412   -0.000470
Length: 120, dtype: float64

In [220]:
chneg['chneg_AR1_innovation']

293         NaN
294    0.001682
295   -0.001713
296    0.002119
297   -0.000085
         ...   
408   -0.000041
409   -0.000643
410   -0.000422
411   -0.000067
412   -0.000496
Name: chneg_AR1_innovation, Length: 120, dtype: float64

In [219]:
px.line(x = chneg['Date'] ,y = - chneg['chneg_AR1_innovation']- (forecast2.predicted_mean - chneg['chneg']), title = 'Difference between results : only computation errors')
